In [362]:
import sys
sys.path.append('/Users/dhairya/cs projects/mdp_course_codes')
import funtion_codes
from funtion_codes import reward, terminal_reward, t_prob

In [363]:
t_matrix = funtion_codes.t_matrix_maker('SUZLON', round_value=2)
exp_rewards = funtion_codes.expected_returns_maker(t_matrix, round_value=2)

In [364]:
exp_rewards

{-10: 0.0,
 -9: 0.01,
 -8: 0.01,
 -7: -0.02,
 -6: -0.003333333333333333,
 -5: 0.009230769230769232,
 -4: 0.006666666666666666,
 -3: -0.00028985507246376827,
 -2: 0.0007983193277310923,
 -1: 0.0007018992568125514,
 0: 0.00041666666666666615,
 1: -0.0003724928366762179,
 2: 0.0006967213114754098,
 3: -0.0012222222222222215,
 4: -0.00034482758620689685,
 5: -0.0005555555555555556,
 6: 0.005,
 7: -0.0025,
 8: -0.006666666666666666,
 9: 0.02,
 10: 0.0,
 20: -0.03}

In [365]:
N = 100 #trading for 10 hours

In [366]:
dp = {}
for i in range(N):
    dp[i] = [[0.0, 0] for _ in range(len(t_matrix)*3)]


In [367]:
mapping = {}
mapping_opp = {}
val = 0
for col in t_matrix:
    for i in {-1, 0, 1}:
        mapping[val] = (col, i)
        mapping_opp[(col, i)] = val
        val+=1


In [368]:
len(dp[0]) == len(mapping)

True

In [369]:
for i in range(len(dp[N-1])):
    dp[N-1][i] = [terminal_reward(state=mapping[i], exp_returns=exp_rewards), 0]

In [370]:
from tqdm import tqdm
for i in tqdm(range(N-1)):
    row_val = N - i - 2
    for j in range(len(dp[row_val])):
        curr_state = mapping[j]
        actions = []

        if curr_state[1] == -1:
            actions = [2, 1, 0]
        elif curr_state[1] == 0:
            actions = [-1, 0, 1]
        elif curr_state[1] == 1:
            actions = [-2, -1, 0]
        
        for action in actions:
            temp_value = reward(state=curr_state, action=action, exp_returns=exp_rewards)
            for k in range(len(dp[row_val])):
                temp_value+= t_prob(next_state=mapping[k], prev_state=curr_state, action=action, t_matrix=t_matrix)*(dp[row_val+1][k][0])
            if temp_value > dp[row_val][j][0]:
                dp[row_val][j][0] = temp_value
                dp[row_val][j][1] = action
        



100%|██████████| 99/99 [00:04<00:00, 24.41it/s]


In [371]:
mapping_opp

{(-10, 0): 0,
 (-10, 1): 1,
 (-10, -1): 2,
 (-9, 0): 3,
 (-9, 1): 4,
 (-9, -1): 5,
 (-8, 0): 6,
 (-8, 1): 7,
 (-8, -1): 8,
 (-7, 0): 9,
 (-7, 1): 10,
 (-7, -1): 11,
 (-6, 0): 12,
 (-6, 1): 13,
 (-6, -1): 14,
 (-5, 0): 15,
 (-5, 1): 16,
 (-5, -1): 17,
 (-4, 0): 18,
 (-4, 1): 19,
 (-4, -1): 20,
 (-3, 0): 21,
 (-3, 1): 22,
 (-3, -1): 23,
 (-2, 0): 24,
 (-2, 1): 25,
 (-2, -1): 26,
 (-1, 0): 27,
 (-1, 1): 28,
 (-1, -1): 29,
 (0, 0): 30,
 (0, 1): 31,
 (0, -1): 32,
 (1, 0): 33,
 (1, 1): 34,
 (1, -1): 35,
 (2, 0): 36,
 (2, 1): 37,
 (2, -1): 38,
 (3, 0): 39,
 (3, 1): 40,
 (3, -1): 41,
 (4, 0): 42,
 (4, 1): 43,
 (4, -1): 44,
 (5, 0): 45,
 (5, 1): 46,
 (5, -1): 47,
 (6, 0): 48,
 (6, 1): 49,
 (6, -1): 50,
 (7, 0): 51,
 (7, 1): 52,
 (7, -1): 53,
 (8, 0): 54,
 (8, 1): 55,
 (8, -1): 56,
 (9, 0): 57,
 (9, 1): 58,
 (9, -1): 59,
 (10, 0): 60,
 (10, 1): 61,
 (10, -1): 62,
 (20, 0): 63,
 (20, 1): 64,
 (20, -1): 65}

In [372]:
from tvDatafeed import TvDatafeed, Interval
username = ''
password = ''

tvl = TvDatafeed(username, password)
ticker = 'SUZLON'
data=tvl.get_hist(ticker, 'NSE', interval=Interval.in_1_hour, n_bars=N+1, fut_contract=None, extended_session=False)

ERROR:tvDatafeed.main:error while signin


In [373]:
data['returns'] = (((data['close'] - data['close'].shift(1))/data['close'].shift(1)).round(2))*100
data = data.dropna()
data['returns'] = data['returns'].astype(int)

In [374]:
data

,symbol,open,high,low,close,volume,returns
datetime,,,,,,,
2025-07-24 14:15:00,NSE:SUZLON,65.26,65.53,65.21,65.45,4844431.0,0
2025-07-24 15:15:00,NSE:SUZLON,65.42,65.49,65.16,65.29,4360679.0,0
2025-07-25 09:15:00,NSE:SUZLON,65.31,65.32,64.08,64.12,11372002.0,-2
2025-07-25 10:15:00,NSE:SUZLON,64.09,64.24,63.84,63.86,5480155.0,0
2025-07-25 11:15:00,NSE:SUZLON,63.86,64.02,63.21,63.24,5674230.0,-1
...,...,...,...,...,...,...,...
2025-08-13 11:15:00,NSE:SUZLON,60.66,60.68,60.10,60.32,16018021.0,-1
2025-08-13 12:15:00,NSE:SUZLON,60.32,60.65,60.31,60.52,7387351.0,0
2025-08-13 13:15:00,NSE:SUZLON,60.51,60.64,60.36,60.44,6497484.0,0


In [375]:
portfolio = 0
reward, action_next = dp[0][mapping_opp[(4, portfolio)]]

In [376]:
data['reward'] = 0.0
data['action_to_do'] = 0
data['portfolio'] = 0

In [377]:
portfolio = 0
for i in range(len(data)-1):
    returns_value = int(data['returns'].iloc[i])
    rewards, action_next = dp[i][mapping_opp[(returns_value, portfolio)]]
    portfolio = portfolio + action_next
    data['portfolio'].iloc[i] = portfolio
    data['reward'].iloc[i] = rewards
    data['action_to_do'].iloc[i] = action_next

/var/folders/wl/q8prsl_x2_g_hl0vv6cf0xj00000gn/T/ipykernel_11505/280348677.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data['portfolio'].iloc[i] = portfolio
/var/folders/wl/q8prsl_x2_g_hl0vv6cf0xj00000gn/T/ipykernel_11505/280348677.p

In [378]:
last_action = -1*int(data['portfolio'].iloc[len(data)-2])

In [379]:
data['action_to_do'].iloc[len(data)-1] = last_action

/var/folders/wl/q8prsl_x2_g_hl0vv6cf0xj00000gn/T/ipykernel_11505/3671441619.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data['action_to_do'].iloc[len(data)-1] = last_action
/var/folders/wl/q8prsl_x2_g_hl0vv6cf0xj00000gn/T/ipykernel_1

In [380]:
r_pnl = 0.0
for i in range(len(data)-1):
    r_pnl += -1*data['action_to_do'].iloc[i]*data['open'].iloc[i+1]
r_pnl += -1*data['action_to_do'].iloc[len(data)-1]*data['close'].iloc[len(data)-1]

In [381]:
r_pnl

np.float64(-12.920000000000044)